## import librairies

In [2]:
from compas_vol.microstructures import TPMSPolar
from compas.geometry import Point, Plane, Frame, Circle, Cylinder, Box
from compas_vol.primitives import VolBox, VolCylinder
from compas_vol.combinations import Intersection, SmoothSubtraction
from compas_vol.modifications import Shell

In [1]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import add_cube

## create volumetric object (CSG tree)

In [4]:
tpmsPol = TPMSPolar(0, 1.2, .0, 6)

In [5]:
vb = VolBox(Box(Frame(Point(0,0,0), [1,0,0], [0,1,0]), 27.5, 23.5, 19.5), 3.)
c = VolCylinder(Cylinder(Circle(Plane([0,0,0], [0,0,1]), 1.0), 23))

In [6]:
ss = SmoothSubtraction(vb, c, 2.0)
i = Shell(Intersection(tpmsPol, ss), 0.75, 1.0)

## workspace (dense grid)

In [3]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -14.0, 14.0
lby, uby = -12.0, 12.0
lbz, ubz = -10.0, 10.0
# resolution(s)
nx, ny, nz = 168, 144, 120
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [7]:
dm = i.get_distance_numpy(x, y, z)

## generate isosurface (marching cube)

In [8]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [9]:
print('use fc slider for cutting plane')
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = add_cube(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))

@mp.interact(fc=(0,1260000))
def facefilter(fc=1260000):
    p.update_object(faces=f[:fc])

use fc slider for cutting plane


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(13.916666…

interactive(children=(IntSlider(value=1260000, description='fc', max=1260000), Output()), _dom_classes=('widge…